In [46]:
import os
import json

from extr_ds.manager.utils.filesystem import load_document

In [47]:
model_checkpoint = 'bert-base-cased'

In [48]:
records = json.loads(load_document(os.path.join('4', 'ents-iob.json')))
record = records[0]

In [49]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [50]:
example = record['tokens']
tokenized_input = tokenizer(example, is_split_into_words=True)

tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens[:10]

['[CLS]', '(', '6', ':', '51', '-', '1st', ')', '(', 'Shot']

In [51]:
tokenized_input.word_ids(batch_index=0)[:10]

[None, 0, 1, 1, 1, 2, 3, 4, 5, 6]

In [52]:
def tokenize_and_align_labels(record, non_token=-100, sub_token=-100):
    tokenized_inputs = tokenizer(
        record['tokens'],
        truncation=True,
        is_split_into_words=True
    )

    labels = []
    for word_idx in tokenized_inputs.word_ids(batch_index=0):
        if word_idx is None:
            labels.append(non_token)

        elif word_idx != previous_word_idx:
            labels.append(record['labels'][word_idx])

        else:
            labels.append(sub_token)

        previous_word_idx = word_idx


    tokenized_inputs['labels'] = labels

    return tokenized_inputs

tokenize_and_align_labels(record)['labels'][:10]

[-100, 'O', 'B-TIME', -100, -100, 'O', 'B-PERIOD', 'O', 'O', 'B-FORMATION']